# Домашнее задание

Домашнее задание состоит из нескольких блоков.


## Эксперименты в ipynb ноутбуках (11 баллов)
- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(5 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(2 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(2 балла)**
- Придумать как можно обработать рекомендации для холодных пользователей. **(2 балла)**

Примечание: за невоспроизводимый код в ноутбуках (например, нарушен порядок выполнения ячеек, вызываются переменные, которые нигде не были объявлены ранее и.т.п) будут штрафы на усмотрение проверяющего.


## Реализация итоговой модели в сервисе (9 баллов)
- Пробитие бейзлайна $MAP@10 \geq 0.074921$ **(6 баллов)**
- Код сервиса соответствует критериям читаемости и воспроизводимости **(3 балла)**





In [71]:
import os
import random

os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
import pandas as pd
import numpy as np
import pickle
import optuna
from functools import partial
import time
from implicit.als import AlternatingLeastSquares
from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import typing as tp
from tqdm import tqdm
from lightfm import LightFM
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization

In [74]:
DATA_PATH = Path("data")

# LOAD DATA

In [201]:
%%time
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

CPU times: user 4.81 s, sys: 2.35 s, total: 7.15 s
Wall time: 7.97 s


# Preprocess

In [76]:
Columns.Datetime = 'last_watch_dt'

In [77]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [78]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [79]:
max_date = interactions[Columns.Datetime].max()

In [80]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [81]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [82]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [83]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [84]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Prepare features and avatars

## User features

In [85]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [86]:
users.fillna('Unknown', inplace=True)

In [87]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [88]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [89]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


### Добавляем 3 аватаров и смотрим их рекомендации

In [90]:
interactions_items = train.merge(items, on=Columns.Item)
interactions_items

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,176549,9506,2021-05-11,4250,72.0,3,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
1,964868,9506,2021-04-30,6725,100.0,3,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
2,450170,9506,2021-07-18,848,14.0,3,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
3,683820,9506,2021-06-28,2617,44.0,3,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
4,547714,9506,2021-08-07,5494,93.0,3,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832706,57116,1289,2021-08-02,623,11.0,3,film,Два билета домой,NaN,2018.0,"драмы, русские",Россия,NaN,18.0,NaN,Дмитрий Месхиев,"Евгений Ткачук, Ирина Рахманова, Ирина Розанов...","Воспитанница детдома Люба узнает, что она не с...","Два, билета, домой, 2018, Россия, месть, отцы,..."
3832707,619743,13516,2021-07-11,14985,27.0,3,series,Звери.,Animals.,2016.0,"анимация, комедии",США,NaN,18.0,HBO,"Майк Лучиано, Уэсли Арчер","Кэтрин Аселтон, Майк Лучиано, Меган О’Нилл, Мэ...",Комедийная анимационная фантазия от создателей...,"Звери, 2016, США"
3832708,336077,14064,2021-05-04,3914,76.0,3,film,Свадьба,Ceremony,2010.0,"зарубежные, комедии, мелодрамы",США,NaN,16.0,NaN,Макс Уинклер,"Джейк Джонсон, Ли Пейс, Майкл Ангарано, Рис То...","Ума Турман, Брук Блум и Рис Томпсон никак не м...","Свадьба, 2010, США, безответная, любовь, друзь..."
3832709,781807,11002,2021-04-10,6750,94.0,3,film,Будь круче!,Be Cool,2005.0,"криминал, музыкальные, зарубежные, мелодрамы, ...",США,NaN,16.0,NaN,Ф. Гэри Грей,"Андре Бенджамин, Винс Вон, Джеймс Вудс, Джон Т...",Американская комедия от режиссера Ф. Гэри Грея...,"Будь, круче, 2005, США, бандиты, гангстеры, му..."


In [91]:
maxid = interactions_items.user_id.unique().max()

### 1st user: Horror Person

In [92]:
horror_interactions = interactions_items[(interactions_items.age_rating >= 18) & (interactions_items.genres.str.contains('ужасы')) & (interactions_items.watched_pct >= 80)]

In [93]:
horror_person_intercations = horror_interactions.item_id.unique()[:5]

In [94]:
horror_person_id = maxid + 1

In [95]:
users = users.append({
    'user_id': horror_person_id,
    'age': 'age_35_44',
    'income': 'income_60_90',
    'sex': 'М',
    'kids_flg': 0
}, ignore_index=True)

In [96]:
def add_interactions(interactions, user_id, item_ids):
    to_append = [{
        'user_id': user_id,
        'item_id': item_id,
        'last_watch_dt': np.datetime64(f'2021-0{random.randint(1,9)}-1{random.randint(0,9)}'),
        'total_dur': 0,
        'weight': 5,
        'watched_pct': random.randint(80, 100),
    } for item_id in item_ids]
    interactions = interactions.append(to_append)
    return interactions

In [97]:
train = add_interactions(train, horror_person_id, horror_person_intercations)

In [98]:
train[train.user_id == horror_person_id]

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,1097558,693,2021-02-12,0,88.0,5
1,1097558,16237,2021-04-18,0,93.0,5
2,1097558,1465,2021-02-18,0,92.0,5
3,1097558,9381,2021-02-14,0,99.0,5
4,1097558,5287,2021-04-16,0,83.0,5


### 2nd user: Person who likes Comedy under 18

In [99]:
comedy_interactions = interactions_items[(interactions_items.age_rating == 16) & (interactions_items.genres.str.contains('комедии')) & (interactions_items.watched_pct >= 80)]

In [100]:
comedy_person_intercations = comedy_interactions.item_id.unique()[:5]

In [101]:
comedy_person_id = maxid + 2

In [102]:
users = users.append({
    'user_id': comedy_person_id,
    'age': 'age_18_24',
    'income': 'income_40_60',
    'sex': 'М',
    'kids_flg': 0
}, ignore_index=True)

In [103]:
train = add_interactions(train, comedy_person_id, comedy_person_intercations)

In [104]:
train[train.user_id == comedy_person_id]

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,1097559,3935,2021-02-13,0,90.0,5
1,1097559,6208,2021-01-17,0,98.0,5
2,1097559,1819,2021-04-16,0,92.0,5
3,1097559,5658,2021-01-17,0,89.0,5
4,1097559,3614,2021-05-17,0,83.0,5


### 3rd user: Mother with kid using her account

In [105]:
mommy_interactions = interactions_items[(interactions_items.age_rating == 0) & (interactions_items.watched_pct >= 80)]

In [106]:
mommy_person_intercations = mommy_interactions.item_id.unique()[:5]

In [107]:
mommy_person_id = maxid + 3

In [108]:
users = users.append({
    'user_id': mommy_person_id,
    'age': 'age_55_64',
    'income': 'income_40_60',
    'sex': 'Ж',
    'kids_flg': 1
}, ignore_index=True)

In [109]:
train = add_interactions(train, mommy_person_id, mommy_person_intercations)

In [110]:
train[train.user_id == mommy_person_id]

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,1097560,9506,2021-06-12,0,100.0,5
1,1097560,8143,2021-07-17,0,93.0,5
2,1097560,14120,2021-07-13,0,82.0,5
3,1097560,16029,2021-01-12,0,92.0,5
4,1097560,10119,2021-08-15,0,80.0,5


In [111]:
avatars = [horror_person_id, comedy_person_id, mommy_person_id]
avatars

[1097558, 1097559, 1097560]

In [112]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
2,721985,Ж,sex
3,704055,Ж,sex
4,1037719,М,sex


# Item features

In [113]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [114]:
items.loc[items[Columns.Item].isin(train[Columns.Item])]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [202]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [116]:
items#.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [117]:
items.nunique()

item_id         14019
content_type        2
title           13454
title_orig       9724
release_year      104
genres           2559
countries         666
for_kids            2
age_rating          6
studios            38
directors        7414
actors          11830
description     13791
keywords        13583
dtype: int64

### Genre

In [118]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Content

In [119]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [120]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [121]:
item_features = pd.concat((genre_feature, content_feature))

In [122]:
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


# Metrics

In [123]:
metrics = {
    'MAP@10': MAP(k=10)
}


# Models

In [124]:
models = {}

In [125]:
%%time
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 1.39 s, sys: 148 ms, total: 1.54 s
Wall time: 1.57 s


In [126]:
TEST_USERS = test[Columns.User].unique()

In [57]:
train.shape

(3832726, 6)

# Optimize hyperparameters with Optuna

In [127]:
def als_model(**kwargs):
    return ImplicitALSWrapperModel(
                    model=AlternatingLeastSquares(
                        factors=kwargs["factors"],
                        random_state=RANDOM_STATE,
                        num_threads=NUM_THREADS,
                    ),
                fit_features_together=kwargs["fit_features_together"],
            )

def lightfm_model(**kwargs):
    return LightFMWrapperModel(
                model=LightFM(
                    no_components=kwargs["no_components"],
                    loss=kwargs["loss"],
                    random_state=RANDOM_STATE,
                    learning_rate=kwargs["learning_rate"],
                    user_alpha=kwargs["user_alpha"],
                    item_alpha=kwargs["item_alpha"],
                ),
                epochs=kwargs["epochs"],
                num_threads=NUM_THREADS,
            )

In [128]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 6

params = {
    'N_FACTORS_ALS': {'values': (30, 45), 'dtype': 'int'}, # ALS
    'N_FACTORS_LFM': {'values': (10, 20), 'dtype': 'int'}, # Lightfm
    'N_EPOCHS': {'values': (1, 3), 'dtype': 'int'}, # Lightfm
    'USER_ALPHA': {'values': (0, 0), 'dtype': 'int'}, # Lightfm
    'ITEM_ALPHA': {'values': (0, 0), 'dtype': 'int'}, # Lightfm
    'LEARNING_RATE': {'values': (0.05, 0.05), 'dtype': 'float'}, # Lightfm
    'lightfm_losses': {'values': ('logistic', 'bpr', 'warp'), 'dtype': 'categorical'}, # Lightfm
    'is_fitting_features': {'values': (True, False), 'dtype': 'categorical'} # ALS
}

reco_algorithms = {
    "ALS": {'model': als_model,
            'params': dict(factors=params["N_FACTORS_ALS"],
                           fit_features_together=params["is_fitting_features"],
                           )
            },
    "LightFM": {'model': lightfm_model,
                'params': dict(no_components=params["N_FACTORS_LFM"],
                               loss=params["lightfm_losses"],
                               learning_rate=params["LEARNING_RATE"],
                               user_alpha=params["USER_ALPHA"],
                               item_alpha=params["ITEM_ALPHA"],
                               epochs=params["N_EPOCHS"],
                               )
                },
}

In [129]:
def objective(trial: optuna.Trial, reco_algorithm):
    reco_model = reco_algorithms[reco_algorithm]['model']
    suggested_params = dict()

    for param_name, param in reco_algorithms[reco_algorithm]['params'].items():
        trial_suggest = getattr(trial, f'suggest_{param["dtype"]}')

        if param["dtype"] != 'categorical':
            suggested_params[param_name] = trial_suggest(param_name, *param['values'])
        else:
            suggested_params[param_name] = trial_suggest(param_name, param['values'])

    model = reco_model(**suggested_params)
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    map10 = calc_metrics(metrics, recos, test, train)["MAP@10"]

    return map10

### Поиск гиперпараметров через Optuna по всем заданным алгоритмам

In [ ]:
optuna_studies = dict()

for i, (algo_name, algorithm) in enumerate(reco_algorithms.items(), 1):
    print(f'Optimizing {i} algorithm with name: {algo_name}')

    study = optuna.create_study(direction='maximize', study_name=algo_name)
    optuna_studies[algo_name] = study

    study.optimize(partial(objective, reco_algorithm=algo_name), n_trials=10)
    time.sleep(1)

    with open(f'data/optuna_studies_{algo_name}_{i}.pkl', 'wb') as fw:
        pickle.dump(optuna_studies, fw, pickle.HIGHEST_PROTOCOL)

[I 2022-12-02 20:43:07,737] A new study created in memory with name: ALS


Optimizing 1 algorithm with name: ALS


[I 2022-12-02 20:57:32,945] Trial 0 finished with value: 0.07403474221754681 and parameters: {'factors': 40, 'fit_features_together': True}. Best is trial 0 with value: 0.07403474221754681.
[I 2022-12-02 21:11:11,794] Trial 1 finished with value: 0.07383735414169257 and parameters: {'factors': 39, 'fit_features_together': True}. Best is trial 0 with value: 0.07403474221754681.
[I 2022-12-02 21:24:50,398] Trial 2 finished with value: 0.07403474221754681 and parameters: {'factors': 40, 'fit_features_together': True}. Best is trial 0 with value: 0.07403474221754681.
[I 2022-12-02 21:33:23,518] Trial 3 finished with value: 0.06405406658470873 and parameters: {'factors': 43, 'fit_features_together': False}. Best is trial 0 with value: 0.07403474221754681.
[I 2022-12-02 21:40:56,953] Trial 4 finished with value: 0.0639795826326622 and parameters: {'factors': 35, 'fit_features_together': False}. Best is trial 0 with value: 0.07403474221754681.
[I 2022-12-02 21:50:41,377] Trial 5 finished with

Optimizing 2 algorithm with name: LightFM


[I 2022-12-03 00:09:35,625] Trial 0 finished with value: 0.00010750743016374326 and parameters: {'no_components': 43, 'loss': 'bpr', 'learning_rate': 0.07029730927196941, 'user_alpha': 9, 'item_alpha': 5, 'epochs': 9}. Best is trial 0 with value: 0.00010750743016374326.
[I 2022-12-03 00:20:54,723] Trial 1 finished with value: 8.299695568532842e-05 and parameters: {'no_components': 41, 'loss': 'bpr', 'learning_rate': 0.018159878451032525, 'user_alpha': 10, 'item_alpha': 6, 'epochs': 3}. Best is trial 0 with value: 0.00010750743016374326.
[I 2022-12-03 00:24:31,127] Trial 2 finished with value: 9.49614019671568e-06 and parameters: {'no_components': 30, 'loss': 'bpr', 'learning_rate': 0.011817414092888478, 'user_alpha': 7, 'item_alpha': 9, 'epochs': 1}. Best is trial 0 with value: 0.00010750743016374326.


### Поиск гиперпараметров через Optuna отдельно на LightFM.

In [56]:
optuna_studies = dict()

for i, (algo_name, algorithm) in enumerate(reco_algorithms.items(), 1):
    if algo_name != 'LightFM':
        continue
    print(f'Optimizing {i} algorithm with name: {algo_name}')

    study = optuna.create_study(direction='maximize', study_name=algo_name)
    optuna_studies[algo_name] = study

    study.optimize(partial(objective, reco_algorithm=algo_name), n_trials=10)
    time.sleep(1)

    with open(f'data/optuna_studies_{algo_name}_{i}.pkl', 'wb') as fw:
        pickle.dump(optuna_studies, fw, pickle.HIGHEST_PROTOCOL)

[I 2022-12-11 12:11:54,086] A new study created in memory with name: LightFM


Optimizing 2 algorithm with name: LightFM


[I 2022-12-11 12:13:58,410] Trial 0 finished with value: 0.07659459039178797 and parameters: {'no_components': 15, 'loss': 'warp', 'learning_rate': 0.05, 'user_alpha': 0, 'item_alpha': 0, 'epochs': 1}. Best is trial 0 with value: 0.07659459039178797.
[I 2022-12-11 12:17:01,857] Trial 1 finished with value: 0.02389820045819892 and parameters: {'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.05, 'user_alpha': 0, 'item_alpha': 0, 'epochs': 1}. Best is trial 0 with value: 0.07659459039178797.
[I 2022-12-11 12:19:37,305] Trial 2 finished with value: 0.0749771541782762 and parameters: {'no_components': 11, 'loss': 'warp', 'learning_rate': 0.05, 'user_alpha': 0, 'item_alpha': 0, 'epochs': 1}. Best is trial 0 with value: 0.07659459039178797.
[I 2022-12-11 12:22:11,738] Trial 3 finished with value: 0.016182758291402838 and parameters: {'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.05, 'user_alpha': 0, 'item_alpha': 0, 'epochs': 3}. Best is trial 0 with value: 0.07659459039178797

KeyboardInterrupt: 

### Как можем видеть, logistic loss и bpr плохо показывают себя на MAP@10, а warp уже побил baseline.

In [67]:
best_params = optuna_studies['LightFM'].best_params
best_params

{'no_components': 15,
 'loss': 'warp',
 'learning_rate': 0.05,
 'user_alpha': 0,
 'item_alpha': 0,
 'epochs': 1}

In [130]:
best_params = {'no_components': 15,
 'loss': 'warp',
 'learning_rate': 0.05,
 'user_alpha': 0,
 'item_alpha': 0,
 'epochs': 1}

In [131]:
reco_model = reco_algorithms['LightFM']['model']
model = reco_model(**best_params)
model.fit(dataset)

In [91]:
model_export_path = 'best_model.pkl'
if not os.path.exists(model_export_path):
    with open(model_export_path, 'wb') as fw:
        pickle.dump(model, fw, pickle.HIGHEST_PROTOCOL)

# ONLINE Approximate Nearest Neighbors

## Qdrant
### С помощью векторного поискового движка qdrant, делаем online инференс с временем ответа ~4ms

In [132]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import numpy as np

In [ ]:
!docker pull qdrant/qdrant

In [27]:
!docker run --name qdrant -d -p 6333:6333 --net=bridge qdrant/qdrant

8f282832e4e88da0022ea30ad69f2faf9c59ac7771a3ba928480a18385c0c8fb


In [133]:
client = QdrantClient(host="localhost", port=6333)

In [134]:
COLLECTION_NAME = 'reco'

In [63]:
with open(f'best_model.pkl', 'rb') as fr:
    model = pickle.load(fr)

In [135]:
def create_collection_and_upload(vectors, payload, ids, col_name, vec_shape, bs, distance: Distance):
    client.recreate_collection(collection_name=col_name,
                               vectors_config=VectorParams(size=vec_shape, distance=distance),
                               on_disk_payload=False)
    client.upload_collection(
        collection_name=col_name,
        vectors=vectors,
        payload=payload,
        ids=ids,
        batch_size=bs,
        parallel=6
    )
    col = client.get_collection(col_name)
    return col


### Аугментируем датасеты юзеров и айтемов

In [136]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [137]:
user_embeddings.shape, item_embeddings.shape

((756565, 17), (14019, 17))

In [138]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()

    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [139]:
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
print(f'augmented_item_embeddings shape {augmented_item_embeddings.shape}')
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
print(f'augmented_user_embeddings shape {augmented_user_embeddings.shape}')

augmented_item_embeddings shape (14019, 18)
augmented_user_embeddings shape (756565, 18)


In [140]:
user_dataset = pd.DataFrame(augmented_user_embeddings, index=dataset.user_id_map.external_ids)
item_dataset = pd.DataFrame(augmented_item_embeddings, index=dataset.item_id_map.external_ids)

In [101]:
with open('data/user_dataset.pkl', 'wb') as fw:
    pickle.dump(user_dataset, fw, pickle.HIGHEST_PROTOCOL)
with open('data/item_dataset.pkl', 'wb') as fw:
    pickle.dump(item_dataset, fw, pickle.HIGHEST_PROTOCOL)

### Создаем коллекцию и грузим вектора айтемов

In [144]:
create_collection_and_upload(item_dataset.to_numpy(),
                             None,
                             ids=item_dataset.index.tolist(),
                             col_name=COLLECTION_NAME,
                             vec_shape=item_dataset.shape[1],
                             bs=6,
                             distance=Distance.DOT
                             )

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=14019, indexed_vectors_count=0, points_count=14019, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=18, distance=<Distance.DOT: 'Dot'>), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=False), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0)), payload_schema={})

In [145]:
client.get_collection('reco')

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=14019, indexed_vectors_count=0, points_count=14019, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=18, distance=<Distance.DOT: 'Dot'>), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=False), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0)), payload_schema={})

### Пример ONLINE ANN поиска по векторам юзеров

In [87]:
%%timeit
query_vector = user_dataset.loc[11542].to_numpy()
hits = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,  # Don't use any filters for now, search across all indexed points
    append_payload=True,  # Also return a stored payload for found points
    with_vectors=False,
    limit=10  # Return N closest points
)
hits = [h.id for h in hits]
hits

3.98 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Avatar reco

In [142]:
avatars

[1097558, 1097559, 1097560]

In [146]:
avatars_reco_item_ids = {}
for avatar in avatars:
    query_vector = user_dataset.loc[avatar].to_numpy()
    hits = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_vector,
        query_filter=None,  # Don't use any filters for now, search across all indexed points
        append_payload=True,  # Also return a stored payload for found points
        with_vectors=False,
        limit=10  # Return N closest points
    )
    hits = [h.id for h in hits]
    avatars_reco_item_ids[avatar] = hits


In [147]:
avatars_reco_item_ids

{1097558: [10440, 15297, 9728, 4151, 142, 13865, 1844, 2657, 3734, 12192],
 1097559: [10440, 4151, 15297, 4880, 9728, 3734, 13865, 2657, 11237, 6809],
 1097560: [15297, 10440, 7571, 9728, 4151, 16166, 3734, 13865, 3182, 7582]}

### Добавляем нормированную колонку с популярностью

In [203]:
item_popularity = interactions.groupby(Columns.Item)['user_id'].agg('count').reset_index().rename(mapper={'user_id': 'popularity'}, axis=1)

In [204]:
item_popularity['popularity'] = ((item_popularity.popularity - item_popularity.popularity.min()) / (item_popularity.popularity.max() - item_popularity.popularity.min())).sort_values()

In [205]:
items = items.merge(item_popularity, on='item_id')

### 1 аватар - Horror Person

#### История просмотров

In [206]:
history_avatar_1 = train[train.user_id == 1097558]
history_avatar_1.merge(items, on='item_id')

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
0,1097558,693,2021-02-12,0,88.0,5,film,Техасская резня бензопилой,The Texas Chain Saw Massacre,1974.0,"зарубежные, ужасы, триллеры, мировая классика",США,NaN,18.0,NaN,Тоуб Хупер,"Аллен Дэнзигер, Гуннар Хансен, Джерри Грин, Дж...","Эталонная классика жанра ужасов, повлиявшая на...","Техасская, резня, бензопилой, 1974, США, друзь...",0.000119
1,1097558,16237,2021-04-18,0,93.0,5,film,Никто не выжил,No One Lives,2012.0,"ужасы, триллеры",США,NaN,18.0,NaN,Рюхэй Китамура,"Люк Эванс, Эделейд Клеменс, Ли Тергесен, Дерек...",Безжалостные преступники спрятали в заброшенно...,"похищение людей, ресторан, мотель, шоссе, поли...",0.007439
2,1097558,1465,2021-02-18,0,92.0,5,series,Кошмары музыкантов,Koshmary muzykantov,2021.0,ужасы,Россия,NaN,18.0,NaN,"Филипп Юрьев, Макс Шишкин, Анджей Гавриш","Хаски, Антон Адасинский, Алишер Моргенштерн, А...",Сборник короткометражных фильмов от российских...,"фильм кошмары музыкантов, кошмары музыкантов м...",0.023121
3,1097558,9381,2021-02-14,0,99.0,5,film,Кто не спрятался,The Rental,2020.0,"драмы, ужасы, триллеры, детективы",США,NaN,18.0,NaN,Дэйв Франко,"Дэн Стивенс, Элисон Бри, Тоби Хасс, Шейла Ванд...",Вы снимали жилье через интернет? Смотрели крас...,"камера, экстази, брат, дом, сталкер, убийство,...",0.013934
4,1097558,5287,2021-04-16,0,83.0,5,film,Брешь,Anti-Life (Breach),2020.0,"боевики, ужасы, фантастика",Канада,NaN,18.0,NaN,Джон Суитц,"Коуди Кирсли, Брюс Уиллис, Рэйчел Николс, Тимо...",2242 год. Земля обречена. Среди 50 миллионов и...,"инопланетянин, меняющий форму, 2020, канада, б...",0.026865


#### Рекомендации

In [207]:
reco_avatar_1 = items[items.item_id.isin(avatars_reco_item_ids[1097558])]
reco_avatar_1.index = reco_avatar_1.item_id
reco_avatar_1.loc[avatars_reco_item_ids[1097558]]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
item_id,,,,,,,,,,,,,,,
10440,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...",1.000000
15297,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...",0.953896
9728,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...",0.656261
4151,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",0.450300
142,142,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма...",0.224078
13865,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...",0.603183
1844,1844,film,Аферистка,I Care A Lot,2020.0,"триллеры, комедии",Великобритания,NaN,18.0,NaN,Джей Блейксон,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...",Марла руководствуется принципом «побеждает сил...,"опекун, аферист, чёрная комедия, мошенник, 202...",0.120392
2657,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ...",0.338740
3734,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",0.369473


#### В рекомендации попали триллеры, боевики, комедии. Совпадение по триллерам по истории просмотров.
#### Ужасы не попали в рекомендации, скорей всего потому что они в целом менее популярные и алгоритм не стал их рекомендовать.

### 2 аватар - Comedy Person

#### История просмотров

In [211]:
history_avatar_1 = train[train.user_id == 1097559]
history_avatar_1.merge(items, on='item_id')

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
0,1097559,3935,2021-02-13,0,90.0,5,film,Бывшая с того света,Blithe Spirit,2020.0,"фэнтези, триллеры, комедии",Великобритания,NaN,16.0,NaN,Эдвард Холл,"Дэн Стивенс, Айла Фишер, Джуди Денч, Лесли Ман...",В поисках вдохновения романист Чарльз Кондомай...,"2020, соединенное королевство, бывшая, того, с...",0.062824
1,1097559,6208,2021-01-17,0,98.0,5,film,Страна чудес Вилли,Willy's Wonderland,2021.0,"ужасы, комедии",США,NaN,16.0,NaN,Кевин Льюис,"Николас Кейдж, Эмили Тоста, Бет Грант, Рик Рай...",У молчаливого мужчины на спортивной машине неп...,"кошмар, карнавал, талисман, злая кукла, одержи...",0.017041
2,1097559,1819,2021-04-16,0,92.0,5,film,Непосредственно Каха,Neposredstvenno Kakha,2020.0,комедии,Россия,NaN,16.0,NaN,Виктор Шамиров,"Артем Карокозян, Артем Калайджян, Марина Калец...",Южнороссийская хитовая комедия по мотивам попу...,"2020, россия, непосредственно, каха",0.058640
3,1097559,5658,2021-01-17,0,89.0,5,film,#Только серьёзные отношения,Serious relationship only,2021.0,"мелодрамы, комедии",Россия,NaN,16.0,NaN,Вячеслав Росс,"Алина Юхневич, Александр Робак, Ирина Пегова, ...",Больше всего Соня Чижик хочет найти настоящую ...,", 2021, россия, только, серьёзные, отношения",0.065535
4,1097559,3614,2021-05-17,0,83.0,5,series,"Отряд ""Хваран""",Hwarang: The Poet Warrior Youth,2016.0,"историческое, комедии",Республика Корея,NaN,16.0,NaN,Юн Сон-щик,"Пак Со-джун, Пак Хён-щик, Ко А-ра, Минхо, То Д...",История об элитном отряде королевства Силла.,"6-й век, боевые искусства, братство, воин, дре...",0.001314


#### Рекомендации

In [210]:
reco_avatar_2 = items[items.item_id.isin(avatars_reco_item_ids[1097559])]
reco_avatar_2.index = reco_avatar_2.item_id
reco_avatar_2.loc[avatars_reco_item_ids[1097559]]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
item_id,,,,,,,,,,,,,,,
10440,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...",1.000000
4151,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",0.450300
15297,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...",0.953896
4880,4880,series,Афера,Afera,2021.0,комедии,Россия,NaN,18.0,NaN,Михаил Старчак,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Смотри:- как кино- как сериалКарантин окончен,...","Афера, Аферисты, Карантин, Пандемия, Карантин ...",0.271871
9728,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...",0.656261
3734,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",0.369473
13865,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...",0.603183
2657,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ...",0.338740
11237,11237,film,День города,NaN,2021.0,комедии,Россия,NaN,16.0,NaN,Алексей Харитонов,"Катерина Шпица, Антон Филипенко, Павел Ворожцо...",Эта история случилась в провинциальном городке...,"2021, россия, день, города",0.127030


#### В рекомендациях видим комедии, триллеры, мелодрамы - совпадает с историей.
#### Также видим и другме жанры, фильмы которых более популярные, поэтмоу попали в рекомендации.


### 3 аватар - Mommy Person

#### История просмотров

In [212]:
history_avatar_1 = train[train.user_id == 1097560]
history_avatar_1.merge(items, on='item_id')

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
0,1097560,9506,2021-06-12,0,100.0,5,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",США,NaN,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн...",0.021219
1,1097560,8143,2021-07-17,0,93.0,5,film,Принц Египта,The Prince of Egypt,1998.0,"драмы, мультфильм, фэнтези",США,NaN,0.0,NaN,"Бренда Чепмен, Стив Хикнер, Саймон Уэллс","Вэл Килмер, Рэйф Файнс, Мишель Пфайффер, Сандр...",Эта история случилась на заре цивилизации в Др...,"моисей, египет, пирамида, исход, царство, мюзи...",0.009760
2,1097560,14120,2021-07-13,0,82.0,5,series,Маша и Медведь,Masha i Medved,2010.0,"мультсериалы, приключения, комедии",Россия,NaN,0.0,NaN,"Олег Кузовков, Олег Ужинов, Денис Червяцов","Алина Кукушкина, Борис Кутневич, Эдуард Назаро...",«Маша и Медведь» - это самый популярный россий...,"6-летний, вокзал, главный герой - женщина, дев...",0.017653
3,1097560,16029,2021-01-12,0,92.0,5,film,Монстр в Париже,Monster in Paris!/Un monster a Paris !,2011.0,"мультфильм, музыкальные, приключения, мелодрам...",Франция,NaN,0.0,NaN,Бибо Бергерон,"Ванесса Паради, Матьё Шедид, Гад Эльмалех, Фра...","Париж. 1910 год. Ужасный монстр, напоминающий ...","париж, франция, монстр, 1900-е, 1910-е, 2010-е...",0.008199
4,1097560,10119,2021-08-15,0,80.0,5,film,Ледниковый период,Ice Age,2002.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,0.0,NaN,"Крис Уэдж, Карлос Салдана","Рэй Романо, Джон Легуизамо, Дэнис Лири, Горан ...",Действие разворачивается 20 тыс. лет назад. Чт...,"человеческая эволюция, потеря любимого, отноше...",0.019426


#### Рекомендации

In [209]:
reco_avatar_3 = items[items.item_id.isin(avatars_reco_item_ids[1097560])]
reco_avatar_3.index = reco_avatar_3.item_id
reco_avatar_3.loc[avatars_reco_item_ids[1097560]]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,popularity
item_id,,,,,,,,,,,,,,,
15297,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...",0.953896
10440,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...",1.000000
7571,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня...",0.140134
9728,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...",0.656261
4151,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",0.450300
16166,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел...",0.069121
3734,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",0.369473
13865,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...",0.603183
3182,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ...",0.058531


#### В рекомендациях преимущественно мультфильмы - совпадает с историей просмотра.
#### Также присутствуют популярные фильмы не по истории просмотра.
#### Помимо этого присутствуют фильмы с рейтингом 18+ - на проде такие скорей всего надо фильтровать для пользователей с детьми.

# Cold recommendations

### Используем подход рекомендаций most popular

In [126]:
most_popular_top_10 = interactions.groupby(Columns.Item).agg('count').sort_values('user_id', ascending=False)[:10].index.to_list()

In [127]:
most_popular_top_10

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [128]:
with open('data/most_popular_top_10.pkl', 'wb') as fw:
    pickle.dump(most_popular_top_10, fw, pickle.HIGHEST_PROTOCOL)

,item_id,user_id
0,0,37
1,1,23
2,2,38
3,3,18
4,4,6
...,...,...
15701,16514,2
15702,16515,1
15703,16516,74
15704,16517,1


In [ ]:
items